# IMDB datasets
This notebook contains the Python-3 script to get IMDB movie data into a database from their downloadable datasets. It assumes you have psycopg2, which you can install from conda. I recommend making a **new** conda environment, so your _root_ environment won't get corrupted. For example, the following command in a conda shell would create a new Python-3.6 environment with psycopg2.

    $ conda create -n py36-imdb python=3.6.3 psycopg2

[psycopg2](http://initd.org/psycopg/) is a binding to the [PostgreSQL](https://www.postgresql.org/) C-API that uses the [Python DB-API 2.0 specification](https://www.python.org/dev/peps/pep-0249/). This means it uses the same syntax to connect and execute queries as the [builtin `sqlite3` Python module](https://docs.python.org/3.6/library/sqlite3.html).

In [1]:
# imports
import csv
import gzip
import os
import urllib
import psycopg2, psycopg2.extras
import logging

# logging
logging.basicConfig()
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.DEBUG)

In [2]:
# meta data about the URL, the data tarballs, and their schema
IMDB_datasets = 'https://datasets.imdbws.com/'
datasets = {
    'title_basics': 'title.basics.tsv.gz',  # information for titles
    'title_crew': 'title.crew.tsv.gz',  # the director and writer information for all the titles in IMDb
    'title_episode': 'title.episode.tsv.gz',  # the tv episode information
    'title_principals': 'title.principals.tsv.gz',
    'title_ratings': 'title.ratings.tsv.gz',
    'name_basics': 'name.basics.tsv.gz'
}
DETAILS = {
    'title_basics': [
        ('tconst', 'TEXT PRIMARY KEY'), ('titleType', 'TEXT'), ('primaryTitle', 'TEXT'),
        ('originalTitle', 'TEXT'), ('isAdult', 'INT'), ('startYear', 'INT'),
        ('endYear', 'INT'), ('runtimeMinutes', 'INT'), ('genres', 'TEXT')
    ],
}

In [3]:
# info to connect to an online database set up for this tutorial
DBNAME = 'breaking-bytes_imdb'
HOST = 'postgresql-breaking-bytes.alwaysdata.net'
PORT = 5432
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')

# connect to the database and get a Python DB-API 2.0 "cursor"
conn = psycopg2.connect(dbname=DBNAME, user=USER, password=PASSWORD, host=HOST, port=PORT)
cur = conn.cursor()

In [4]:
# create title.basic table
EXPR = ('CREATE TABLE title_basics(%s, %s, %s, %s, %s, %s, %s, %s, %s);'
        % tuple(' '.join(kv) for kv in DETAILS['title_basics']))
cur.execute(EXPR)
conn.commit()

In [5]:
# get the 'title.basics' dataset
with urllib.request.urlopen(IMDB_datasets + datasets['title_basics']) as http_response:
    tsv = gzip.decompress(http_response.read())

In [6]:
# split the lines into rows
rows = tsv.decode('utf-8').split('\n')

LOGGER.debug(rows[0].split('\t'))  # take a look at the headers

# skip the first row with the headers and the last row with only a newline
rows = rows[1:-1]

# split the rows into columns
records = [row.split('\t') for row in rows]

# replace IMDB's NULL, "\\N", which None 
records = [tuple(None if r == '\\N' else r for r in record)
           for record in records]

DEBUG:__main__:['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']


In [7]:
# copy this data to the database
COUNT = len(records)
LOOPS = 100
CHUNKS = COUNT // LOOPS

idx = 0
for n in range(LOOPS):
    jdx = idx + CHUNKS
    LOGGER.debug('loop: %d, start: %d, stop: %d', n, idx, jdx)
    psycopg2.extras.execute_values(cur, 'INSERT INTO title_basics VALUES %s;', records[idx:jdx])
    conn.commit()
    idx += CHUNKS

DEBUG:__main__:loop: 0, start: 0, stop: 48137
DEBUG:__main__:loop: 1, start: 48137, stop: 96274
DEBUG:__main__:loop: 2, start: 96274, stop: 144411
DEBUG:__main__:loop: 3, start: 144411, stop: 192548
DEBUG:__main__:loop: 4, start: 192548, stop: 240685
DEBUG:__main__:loop: 5, start: 240685, stop: 288822
DEBUG:__main__:loop: 6, start: 288822, stop: 336959
DEBUG:__main__:loop: 7, start: 336959, stop: 385096
DEBUG:__main__:loop: 8, start: 385096, stop: 433233
DEBUG:__main__:loop: 9, start: 433233, stop: 481370
DEBUG:__main__:loop: 10, start: 481370, stop: 529507
DEBUG:__main__:loop: 11, start: 529507, stop: 577644
DEBUG:__main__:loop: 12, start: 577644, stop: 625781
DEBUG:__main__:loop: 13, start: 625781, stop: 673918
DEBUG:__main__:loop: 14, start: 673918, stop: 722055
DEBUG:__main__:loop: 15, start: 722055, stop: 770192
DEBUG:__main__:loop: 16, start: 770192, stop: 818329
DEBUG:__main__:loop: 17, start: 818329, stop: 866466
DEBUG:__main__:loop: 18, start: 866466, stop: 914603
DEBUG:__main

In [8]:
cur.close()
conn.close()

In [14]:
print(COUNT)

4813797
